<center><h1>NeuroM Tutorial</h1></center>


You can take a look to the online guide here - [NEUROM.](https://developer.humanbrainproject.eu/docs/projects/neurom/0.0.3/readme.html)

The first __IMPORTANT__ thing you should do is:<br>
to download all the files from the folder "morphologies"
in the MOODLE and save them in the Storage of the collab.

First we will go throught the tutorial with one example file.<br>
ENJOY IT!!

In [ ]:
# Make sure that all the plots will be shown
%matplotlib inline 

# be sure that all of us use the same neurom version
!pip install neurom==1.4.5

# import the packages
import pylab as plt
import neurom as nm

In [ ]:
# These lines allow us to take and work with the desired files in the storage.
# Let's start with one example called "cell21.CNG.swc"
collab_path = get_collab_storage_path()
path_to_neuron = '/tmp/downloaded_exampleCell.CNG.swc'
clients = get_hbp_service_client()

# download file from the storage to the virtual machine of the collab
clients.storage.download_file(collab_path + '/exampleCell.CNG.swc', path_to_neuron)

# 1. Load Morphologies & Internal Representation

In [ ]:
# compatible file formats for morphology could be: ascii, swc, hdf5

neuron = nm.load_neuron(path_to_neuron)
print neuron.name

A morphology object consists of:
* Soma
* Neurites (basal dendrites, axon, apical dendrites)
* Sections

In [ ]:
# A soma is represented as a set of points (x, y, z, radius)

print "Soma Points: \n", neuron.soma.points
print "Soma Center: \n", neuron.soma.center
print "Soma Radius: \n", neuron.soma.radius

In [ ]:
neurites = neuron.neurites

print neurites

# enumerate returns the number of the list item and the object itself
# e.g. (1, neurite1), (2, neurite2), etc.
for i, neurite in enumerate(neurites):
    
    # iter_sections is an iterator. It returns the values one by one every time it is called
    # therefore by applying the list function we force to return all the values.
    number_of_sections = len(list(neurite.iter_sections()))
    
    # points is a numpy array, where instead of len, shape is used
    # first element is the number of points, second the number of coordinates and a radius (4)
    number_of_points = neurite.points.shape[0]

    # strings to use for printing (\n = line change)
    string_neurite = "Neurite {0}:\n".format(i)
    string_type    = "Type: {0}\n".format(neurite.type)
    string_n_secs  = "Number of Sections: {0}\n".format(number_of_sections)
    string_n_pnts  = "Number of Points: {0}\n".format(number_of_points)
    
    # join concatenates strings in a list by linking them with the provided string (here a tab identation \t)
    print "\t".join((string_neurite, string_type, string_n_secs, string_n_pnts))


The morphology of each neurite is represented as a tree of sections.

* A segment is the line that connects two points
* A section is a set of points that form a piecewise linear curve (i.e. it's a set of connected segments).
* A section starts from the root (start of the tree) or a branching point and ends at another branching point or at a termination of the tree



In [ ]:
# the first section of the tree
root_section = neurite.root_node

# root section means it's the first one from the soma -> No Parent
print "Root section parent: \n\t", root_section.parent  
print "Root section children: \n\t", root_section.children

# to get all the sections, we have to traverse the tree
# see tree traversal (Pre-Order): https://en.wikipedia.org/wiki/Tree_traversal
sections = list(neurite.iter_sections())[:20]

print "\n",sections

# 2. View your morphologies

In [ ]:
from neurom import viewer

help(viewer.draw)

In [ ]:
# planes: 'xy', 'xz', 'yz'
figure, axis = viewer.draw(neuron, mode='2d', plane='xy')

# the axis handler is returend for further customization
axis.set_title('My Favorite Neuron')
axis.set_xlabel('micrometers (um)')
axis.set_ylabel('micrometers (um)')

In [ ]:
viewer.draw(neuron.soma, plane='xy')

And something for the adventurous.
To work with subplots and create more complex plots, we need to use lower level plot functions that reside in the view module

In [ ]:
from neurom.view import view
from neurom.view.common import update_plot_limits

N = len(neurites)

# create N subplots, one for each neurite
f, axes = plt.subplots(1, N, figsize=(15, 3)) # figsize in inches (width, height)

for i, neurite in enumerate(neurites):
    current_axes = axes[i]
    
    # draw the neurite in the i-th subplot
    # new_fig set to False will prevent the creation of a new figure
    # every time the function is called. The one that we created above will be used instead
    view.plot_tree(current_axes, neurite, plane='xy')
    update_plot_limits(current_axes, white_space=10)
    # remove the xy axes for a prettier result
    current_axes.axis('off')

    # remove the title for each subplot
    current_axes.set_title('')

# set a global title for the figure
f.suptitle(neuron.name, fontsize=30)



# 3. Extract basic morphometrics (features)

There are two types of features, the neuron and neurite features.

Neurite features can be extracted from a single neurite, a list of neurites or a neuron.

Neuron features can only be extracted from a neuron, because some additional info is ussuale required (e.g. distance from soma).

All features can be called through the neurom.get function

In [ ]:
help(nm.get)

In [ ]:
# extract section_lengths
section_lengths_neuron = nm.get('section_lengths', neuron)
print "Section lengths: \n", section_lengths_neuron[:10]

# Extract the local bifurcation angles
local_bif_angles = nm.get('local_bifurcation_angles', neuron)
print "Section local bifurcation angles: \n", local_bif_angles[:10]

In [ ]:
try:
    # this must err because no soma is available
    nm.get('soma_radii', neuron.neurites)

except AttributeError:
    
    print "Soma was not found. Input object is wrong."


## Select Neurite Types

The previous examples treated all neurites in the same way. NeuroM allows you to extract morphometrics for a selected type of trees.

In [ ]:
dir(nm.NeuriteType)

In [ ]:
# Extract the section lengths of axonal trees
ax_section_lengths = nm.get('section_lengths', neuron, neurite_type=nm.NeuriteType.axon)

# Extract the section lengths of basal dendrite trees
ba_section_lengths = nm.get('section_lengths', neuron, neurite_type=nm.NeuriteType.basal_dendrite)

# Extract the section lengths of apical dendrite trees
ap_section_lengths = nm.get('section_lengths', neuron, neurite_type=nm.NeuriteType.apical_dendrite)

print '\naxonal ', ax_section_lengths
print '\nbasal  ', ba_section_lengths
print '\napical ', ap_section_lengths

## Let's plot the morphometrics

In [ ]:
def histogram(values):
    """Generates a histogram in a new figure"""
    
    # create an image with one subplot
    f, ax = plt.subplots(1,1, figsize=(5,5))
    
    # create a histogram with specified bins and normalized
    ax.hist(values, bins=10, normed=True)
    
    ax.set_title(feature.replace('_', ' ')) # replace the underscores with spaces in the string
    
    ax.set_xlabel('units')
    ax.set_ylabel('density')
    
    return f, ax

def boxplot(data):
    '''Generates a boxplot in a new figure'''

    # create an image with one subplot
    f, ax = plt.subplots(1,1, figsize=(5,5))

    ax.boxplot(data) # plot boxplot
    
    ax.set_ylabel('values')

    return f, ax

feature =  'section_lengths'
values = nm.get(feature, neuron)

histogram(values)
boxplot(values)

# 4. Check Morphologies for errors

In [ ]:
# the structural checks concern the valid structure of a morphology
# the neuron_checks are more specific to simulation
# options set the tolerance parameters for the checks

config = {
    'checks': {
        'structural_checks': [
            'is_single_tree',
            'has_valid_soma',
            'has_soma_points',
            'has_sequential_ids',
            'has_increasing_ids',
            'has_valid_neurites',
            'no_missing_parents',
            'has_all_finite_radius_neurites'
        ],
        'neuron_checks': [
            'has_axon',
            'has_basal_dendrite',
            'has_apical_dendrite',
            'has_no_jumps',
            'has_no_fat_ends',
            'has_nonzero_soma_radius',
            'has_all_nonzero_neurite_radii',
            'has_all_nonzero_section_lengths',
            'has_all_nonzero_segment_lengths',
        ]
    },
    'options': {
        'has_nonzero_soma_radius': 0.0,
        "has_all_nonzero_neurite_radii": 0.007,
        "has_all_nonzero_segment_lengths": 0.01,
        "has_all_nonzero_section_lengths": 0.01,
    }
}

In [ ]:
from neurom.check.runner import CheckRunner

# create a CheckRunner object by providing the configuration dict
check_runner = CheckRunner(config)

# run on a directory or a morphology
results = check_runner.run(path_to_neuron) 

for morph in results['files']:
    
    print "Morphology: {} \n".format(morph)
    
    checks_dict = results['files'][morph]
    
    # empty strings for our results
    passed = ""
    failed = ""
    
    # iterate over dictionary's items
    for check_name, status in checks_dict.items():
        
        # the string with the check's name and status
        string = "\t{0: <50} : {1}\n".format(check_name, status)
    
        # separate passed from failed checks
        if status is True:
            
            passed += string # concatenates to the current passed string
            
        else:
            
            failed += string # concatenats to the current failed string

    print passed
    print failed

___
__Exercise:__ Work with the other morphology files that you have uploaded to the collab Storage, <br> 
some of them are good and some of them have errors.<br>
Find the files with errors and enumerate the errors in each file. Try to plot them and look at the morphology errors.
___